In [ ]:
import tkinter.messagebox
import tkinter.ttk
from tkinter import *
import joblib
from PIL import Image, ImageTk
from functools import partial
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings(action = 'ignore')



class Error(Exception):
    """Base class for other exceptions"""
    pass


class InvalidError(Error):
    """Raised when the percentage of targets hit is greater than 100%"""
    pass


class EmptyCellsError(Error):
    """Raised when at least one of the required input fields is empty"""
    pass


class MainPage(Frame):
    """Main page when the GUI is ran. Allows the user to select customer or employee"""

    def __init__(self, master):
        self.master = master
        self.frame = Frame(self.master)
        self.label = Label(self.master, image=back_pic)
        self.label.place(x=0, y=0, relwidth=1, relheight=1)
        self.title_label = Label(self.master, font=("Helvetica", 16),
                                 text="Welcome to VirtualRehab Machine Learning. Please select whether"
                                      " you want Table1 or Table2:").pack(pady='3')
        self.table1_button = Button(self.master, font=("Helvetica", 16), text="Table1", bg="SkyBlue1",
                                  command=self.table1_window).pack(pady='3')
        self.table2_button = Button(self.master, font=("Helvetica", 16), text="Table2", bg="NavajoWhite3",
                                 command=self.table2_window).pack(pady='3')
        self.quit = Button(self.master, font=("Helvetica", 12), text="Quit", bg="black", fg='white',
                           command=self.master.destroy).pack(pady='3')
        self.frame.pack()

    def table1_window(self):
        self.newWindow = Toplevel(self.master)
        self.newWindow.geometry("%dx%d" % (width, height))
        self.master.withdraw()
        self.app = Table1(self.newWindow)


    def table2_window(self):
        self.newWindow = Toplevel(self.master)
        self.newWindow.geometry("%dx%d" % (width, height))
        self.master.withdraw()
        self.app = Table2(self.newWindow)



class Table1(MainPage):

    def __init__(self, master):
        self.temp_difficulty = 'Easy'
        self.master = master
        self.frame = Frame(self.master)
        self.label = Label(self.master, image=back_pic)
        self.label.place(x=0, y=0, relwidth=1, relheight=1)
        self.frame.grid(row=0, column=0)

        self.title = Label(self.master, text="Please enter the initial assessment details below:", font=("Helvetica", 16)).grid(row=0,
                                                                                                               column=0,
                                                                                                               columnspan=3,
                                                                                                               pady='3')

        self.first = Label(self.master, text="Average Shoulder Abduction:", width='30').grid(row=1, column=0, pady='3')
        self.shoulder_abduction = Entry(self.master, width='5')
        self.shoulder_abduction.insert(END, 100)
        self.shoulder_abduction.grid(row=1, column=1, pady='3')

        self.second = Label(self.master, text="Average Shoulder Flexion:", width='30').grid(row=2, column=0, pady='3')
        self.shoulder_flexion = Entry(self.master, width='5')
        self.shoulder_flexion.insert(END, 80)
        self.shoulder_flexion.grid(row=2, column=1, pady='3')

        self.third = Label(self.master, text="Average Elbow Flexion:", width='30').grid(row=3, column=0, pady='3')
        self.elbow_flexion = Entry(self.master, width='5')
        self.elbow_flexion.insert(END, 90)
        self.elbow_flexion.grid(row=3, column=1, pady='3')

        self.fourth = Label(self.master, text="Total Time Taken:", width='30').grid(row=4, column=0, pady='3')
        self.time_taken = Entry(self.master, width='5')
        self.time_taken.insert(END, 110)
        self.time_taken.grid(row=4, column=1, pady='3')

        self.update = Button(self.master, text="Output Result", font=("Helvetica", 16), bg="green", 
                             command=self.change).grid(row=7, columnspan=2, pady='3')

        self.back = Button(self.master, font=("Helvetica", 12), text="Back", fg="white", bg='black',
                           command=self.back).grid(row=8, column=0, pady='3')
        self.other = Button(self.master, font=("Helvetica", 12), text="Table2", fg="white", bg='black',
                          command=partial(self.switch, self.temp_difficulty)).grid(row=8, column=1, pady='3')

    def change(self):
        sh_abd = self.shoulder_abduction.get()
        sh_flx = self.shoulder_flexion.get()
        el_flx = self.elbow_flexion.get()
        tt = self.time_taken.get()

        try:
            if not sh_abd or not sh_flx or not el_flx or not tt:
                raise EmptyCellsError
            sh_abd = int(sh_abd)
            sh_flx = int(sh_flx)
            el_flx = int(el_flx)
            tt = int(tt)
            if sh_abd < 0 or sh_flx < 0 or el_flx < 0 or tt < 0:
                raise ValueError

        except EmptyCellsError:
            tkinter.messagebox.showinfo("---- ERROR ----", "Please fill in all of the cells.", icon="warning")
        except InvalidError:
            tkinter.messagebox.showinfo("---- ERROR ----", "Invalid format!", icon="warning")
        except ValueError:
            tkinter.messagebox.showinfo("---- ERROR ----", "Invalid cell format!", icon="warning")
        else:
            if sh_abd < 55 or sh_flx < 55 or el_flx < 55:
                num = 1
            elif sh_abd > 120 and sh_flx > 90 and el_flx > 90:
                num = 3
            else:
                pred_new = table1_rf.predict([[sh_abd, sh_flx, el_flx, tt]])
                num = pred_new[0]
            if num == 1:
                output = 'Easy'
            if num == 2:
                output = 'Medium'
            if num == 3:
                output = 'Hard'
            self.temp_difficulty = output
            tkinter.messagebox.showinfo("---- OUTPUT ----", f"Output for Table1 is {self.temp_difficulty}.",
                                        icon="info")

    def back(self):
        self.newWindow = Toplevel(self.master)
        self.newWindow.geometry("%dx%d" % (width, height))
        self.app = MainPage(self.newWindow)
        self.master.withdraw()
        
    def switch(self, temp_difficulty):
        self.newWindow = Toplevel(self.master)
        self.newWindow.geometry("%dx%d" % (width, height))
        self.master.withdraw()
        self.app = Table2(self.newWindow, self.temp_difficulty)
        
        
class Table2(Table1):

    def __init__(self, master, *args):
        self.master = master
        self.frame = Frame(self.master)
        self.label = Label(self.master, image=back_pic)
        self.label.place(x=0, y=0, relwidth=1, relheight=1)
        self.frame.grid(row=0, column=0)
        if args:
            self.imported_difficulty = args[0]
        else:
            self.imported_difficulty = 'Easy'

        self.title = Label(self.master, text="Please enter the initial game performance details below:", font=("Helvetica", 16)).grid(row=0,
                                                                                                               column=0,
                                                                                                               columnspan=3,
                                                                                                               pady='3')
        
        self.first = Label(self.master, text="Difficulty:", width='30').grid(row=1, column=0, pady='3')
        self.dif = StringVar(self.master)
        self.dif.set(self.imported_difficulty)  # default value
        self.difficulty = OptionMenu(master, self.dif, 'Easy', 'Medium', 'Hard')
        self.difficulty.grid(row=1, column=1, pady='3')

        self.second = Label(self.master, text="Total Time Taken:", width='30').grid(row=2, column=0, pady='3')
        self.time_taken = Entry(self.master, width='5')
        self.time_taken.insert(END, 150)
        self.time_taken.grid(row=2, column=1, pady='3')

        self.third = Label(self.master, text="Percentage of Targets Hit:", width='30').grid(row=3, column=0, pady='3')
        self.percentage = Entry(self.master, width='5')
        self.percentage.insert(END, 70)
        self.percentage.grid(row=3, column=1, pady='3')

        self.fourth = Label(self.master, text="Average (Successful) Hit Time:", width='30').grid(row=4, column=0, pady='3')
        self.hit_time = Entry(self.master, width='5')
        self.hit_time.insert(END, 4.5)
        self.hit_time.grid(row=4, column=1, pady='3')


        self.update = Button(self.master, text="Output Result", command=self.change, font=("Helvetica", 16), 
                             bg="green").grid(row=7, columnspan=2, pady='3')

        self.back = Button(self.master, font=("Helvetica", 12), text="Back", fg="white", bg='black',
                           command=self.back).grid(row=8, column=0, pady='3')
        self.other = Button(self.master, font=("Helvetica", 12), text="Table1", fg="white", bg='black',
                          command=self.switch).grid(row=8, column=1, pady='3')

    def change(self):
        diff = self.dif.get()
        tt = self.time_taken.get()
        perc = self.percentage.get()
        hit = self.hit_time.get()

        try:
            if not tt or not perc or not hit:
                raise EmptyCellsError
            tt = float(tt)
            perc = int(perc)
            perc /= 100
            hit = float(hit)
            if tt < 0 or perc < 0 or hit < 0:
                raise ValueError
            if perc > 1:
                raise InvalidError

        except EmptyCellsError:
            tkinter.messagebox.showinfo("---- ERROR ----", "Please fill in all of the cells.", icon="warning")
        except InvalidError:
            tkinter.messagebox.showinfo("---- ERROR ----", "Percentage too high!", icon="warning")
        except ValueError:
            tkinter.messagebox.showinfo("---- ERROR ----", "Invalid cell format!", icon="warning")
        else:
            if perc < 0.5:
                tkinter.messagebox.showinfo("---- OUTPUT ----", f"Output for Table2 is {diff}{1}.",
                                        icon="info")
            elif perc >= 0.95:
                tkinter.messagebox.showinfo("---- OUTPUT ----", f"Output for Table2 is {diff}{5}.",
                                        icon="info")
            else:
                if diff == 'Easy':
                    pred_new = table2_rf_easy.predict([[tt, perc, hit]])
                    output = pred_new[0]
                if diff == 'Medium':
                    pred_new = table2_rf_medium.predict([[tt, perc, hit]])
                    output = pred_new[0]
                if diff == 'Hard':
                    pred_new = table2_rf_hard.predict([[tt, perc, hit]])
                    output = pred_new[0]
                tkinter.messagebox.showinfo("---- OUTPUT ----", f"Output for Table2 is {diff}{output}.",
                                            icon="info")

        
    def switch(self):
        self.newWindow = Toplevel(self.master)
        self.newWindow.geometry("%dx%d" % (width, height))
        self.master.withdraw()
        self.app = Table1(self.newWindow)



def main():
    global table1_rf, table2_rf_easy, table2_rf_medium, table2_rf_hard, width, height
    table1_rf = joblib.load('Table1_random_forest.pkl')
    table2_rf_easy = joblib.load('Table2_random_forest_easy.pkl')
    table2_rf_medium = joblib.load('Table2_random_forest_medium.pkl')
    table2_rf_hard = joblib.load('Table2_random_forest_hard.pkl')
    window = Tk()
    window.title("VirtualRehab Machine Learning")
    width = window.winfo_screenwidth()
    height = window.winfo_screenheight()
    window.geometry("%dx%d" % (width, height))
#     window.geometry('1096x720')
    image = Image.open("Background.png")
    image = image.resize((width, height))
    global back_pic
    back_pic = ImageTk.PhotoImage(image)
    label = Label(window, image=back_pic)
    label.place(x=0, y=0, relwidth=1, relheight=1)
    app = MainPage(window)
    window.mainloop()


if __name__ == '__main__':
    main()
